In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import chi2, norm

# تلاش برای import کتابخانه‌های مورد نیاز
try:
    from sklearn.metrics import cohen_kappa_score
    SKLEARN_AVAILABLE = True
except ImportError:
    SKLEARN_AVAILABLE = False

try:
    from statsmodels.stats.contingency_tables import mcnemar
    from statsmodels.stats.proportion import proportions_chisquare_trendindependence
    from statsmodels.stats.power import TTestPower, TTestIndPower, GofChisquarePower
    STATSMODELS_AVAILABLE = True
except ImportError:
    STATSMODELS_AVAILABLE = False

try:
    from lifelines.statistics import logrank_test
    from lifelines import CoxPHFitter
    LIFELINES_AVAILABLE = True
except ImportError:
    LIFELINES_AVAILABLE = False


# ------------------------------------------------------------
# آزمون مک‌نمار (McNemar's test)
# ------------------------------------------------------------
class McNemarTest:
    def __init__(self, table, correction=True):
        """
        table: جدول 2×2 به صورت [[a,b],[c,d]]
        correction: اعمال تصحیح پیوستگی (Yates)
        """
        self.table = np.asarray(table)
        if self.table.shape != (2, 2):
            raise ValueError("جدول باید 2×2 باشد")
        self.correction = correction
        self._compute()

    def _compute(self):
        if STATSMODELS_AVAILABLE:
            result = mcnemar(self.table, exact=False, correction=self.correction)
            self.statistic = result.statistic
            self.p_value = result.pvalue
        else:
            # پیاده‌سازی دستی
            a, b, c, d = self.table[0,0], self.table[0,1], self.table[1,0], self.table[1,1]
            if self.correction:
                self.statistic = (abs(b - c) - 1) ** 2 / (b + c)
            else:
                self.statistic = (b - c) ** 2 / (b + c)
            self.p_value = 1 - chi2.cdf(self.statistic, 1)

    def summary(self, alpha=0.05):
        print("آزمون مک‌نمار (McNemar's test)")
        print("جدول 2×2:")
        print(f"  [{self.table[0,0]}, {self.table[0,1]}]")
        print(f"  [{self.table[1,0]}, {self.table[1,1]}]")
        print(f"آماره کای‌دو = {self.statistic:.4f}, درجه آزادی = 1")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر (عدم توافق حاشیه‌ای) در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر (توافق حاشیه‌ای) در سطح {alpha}")


# ------------------------------------------------------------
# کاپای کوهن (Cohen's kappa)
# ------------------------------------------------------------
class CohensKappa:
    def __init__(self, y1, y2, weights=None):
        """
        y1, y2: برچسب‌های دو ارزیاب (آرایه‌های یکسان)
        weights: 'linear', 'quadratic', یا None
        """
        self.y1 = np.asarray(y1)
        self.y2 = np.asarray(y2)
        if len(self.y1) != len(self.y2):
            raise ValueError("طول دو بردار باید برابر باشد")
        self.weights = weights
        self._compute()

    def _compute(self):
        if SKLEARN_AVAILABLE:
            self.kappa = cohen_kappa_score(self.y1, self.y2, weights=self.weights)
        else:
            # پیاده‌سازی ساده دستی (بدون وزن)
            # ساخت ماتریس توافق
            n = len(self.y1)
            classes = np.unique(np.concatenate([self.y1, self.y2]))
            k = len(classes)
            obs = np.zeros((k, k))
            for i in range(n):
                idx1 = np.where(classes == self.y1[i])[0][0]
                idx2 = np.where(classes == self.y2[i])[0][0]
                obs[idx1, idx2] += 1
            obs /= n
            row_marg = obs.sum(axis=1)
            col_marg = obs.sum(axis=0)
            exp = np.outer(row_marg, col_marg)
            po = np.trace(obs)
            pe = np.trace(exp)
            self.kappa = (po - pe) / (1 - pe) if pe != 1 else 0

    def summary(self):
        print("کاپای کوهن (Cohen's kappa)")
        print(f"تعداد نمونه: {len(self.y1)}")
        print(f"کاپا = {self.kappa:.4f}")
        if self.kappa < 0:
            print("تفسیر: توافق کمتر از شانس")
        elif self.kappa < 0.2:
            print("تفسیر: توافق ضعیف")
        elif self.kappa < 0.4:
            print("تفسیر: توافق نسبتاً ضعیف")
        elif self.kappa < 0.6:
            print("تفسیر: توافق متوسط")
        elif self.kappa < 0.8:
            print("تفسیر: توافق خوب")
        else:
            print("تفسیر: توافق عالی")


# ------------------------------------------------------------
# آزمون روند کاکرن-آرمیتاژ (Cochran–Armitage test for trend)
# ------------------------------------------------------------
class CochranArmitageTest:
    def __init__(self, counts, scores):
        """
        counts: آرایه 2×k از تعداد موفقیت‌ها و کل (یا دو آرایه)
        scores: نمرات (اسکور) برای هر سطح
        """
        counts = np.asarray(counts)
        if counts.ndim == 2 and counts.shape[0] == 2:
            self.success = counts[0, :]
            self.total = counts[1, :]
        elif counts.ndim == 2 and counts.shape[1] == 2:
            self.success = counts[:, 0]
            self.total = counts[:, 1]
        else:
            raise ValueError("counts باید 2×k یا k×2 باشد")
        self.scores = np.asarray(scores)
        if len(self.success) != len(self.scores):
            raise ValueError("تعداد سطوح باید با تعداد نمرات برابر باشد")
        self._compute()

    def _compute(self):
        if STATSMODELS_AVAILABLE:
            # استفاده از statsmodels
            chi2, p_value, _ = proportions_chisquare_trendindependence(self.success, self.total, self.scores)
            self.statistic = chi2
            self.p_value = p_value
        else:
            # پیاده‌سازی دستی
            n = self.total
            p = self.success / n
            p_bar = np.sum(self.success) / np.sum(self.total)
            # نمرات
            s = self.scores
            s_bar = np.sum(n * s) / np.sum(n)
            # واریانس
            var = p_bar * (1 - p_bar) * np.sum(n * (s - s_bar)**2)
            # آماره
            num = np.sum(n * (s - s_bar) * (p - p_bar))
            self.statistic = num**2 / var
            self.p_value = 1 - chi2.cdf(self.statistic, 1)

    def summary(self, alpha=0.05):
        print("آزمون روند کاکرن-آرمیتاژ (Cochran–Armitage test for trend)")
        print(f"آماره کای‌دو = {self.statistic:.4f}, درجه آزادی = 1")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: روند معنی‌دار در سطح {alpha}")
        else:
            print(f"نتیجه: روند معنی‌دار نیست در سطح {alpha}")


# ------------------------------------------------------------
# آزمون رتبه‌ای لگاریتمی (Log-rank test)
# ------------------------------------------------------------
class LogRankTest:
    def __init__(self, durations, events, groups):
        """
        durations: زمان‌های بقا
        events: رویداد (1 اگر رویداد رخ داده، 0 اگر سانسور شده)
        groups: برچسب گروه (دو گروه)
        """
        self.durations = np.asarray(durations)
        self.events = np.asarray(events)
        self.groups = np.asarray(groups)
        if not LIFELINES_AVAILABLE:
            raise ImportError("برای اجرای این آزمون، lifelines باید نصب باشد: pip install lifelines")

    def _compute(self):
        unique_groups = np.unique(self.groups)
        if len(unique_groups) != 2:
            raise ValueError("فقط دو گروه پشتیبانی می‌شود")
        group1 = unique_groups[0]
        group2 = unique_groups[1]
        idx1 = self.groups == group1
        idx2 = self.groups == group2
        result = logrank_test(self.durations[idx1], self.durations[idx2],
                              event_observed_A=self.events[idx1],
                              event_observed_B=self.events[idx2])
        self.statistic = result.test_statistic
        self.p_value = result.p_value

    def summary(self, alpha=0.05):
        self._compute()
        print("آزمون رتبه‌ای لگاریتمی (Log-rank test)")
        print(f"آماره = {self.statistic:.4f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: تفاوت معنی‌دار بین منحنی‌های بقا در سطح {alpha}")
        else:
            print(f"نتیجه: عدم تفاوت معنی‌دار بین منحنی‌های بقا در سطح {alpha}")


# ------------------------------------------------------------
# آزمون مدل خطرات متناسب کاکس (Cox proportional hazards model test)
# ------------------------------------------------------------
class CoxModelTest:
    def __init__(self, df, duration_col, event_col, covariates):
        """
        df: DataFrame شامل داده‌ها
        duration_col: نام ستون زمان بقا
        event_col: نام ستون رویداد
        covariates: لیست اسامی متغیرهای پیش‌بین
        """
        if not LIFELINES_AVAILABLE:
            raise ImportError("برای اجرای این آزمون، lifelines باید نصب باشد: pip install lifelines")
        import pandas as pd
        if not isinstance(df, pd.DataFrame):
            raise ValueError("داده باید DataFrame باشد")
        self.df = df
        self.duration_col = duration_col
        self.event_col = event_col
        self.covariates = covariates
        self._fit()

    def _fit(self):
        self.cph = CoxPHFitter()
        self.cph.fit(self.df, duration_col=self.duration_col, event_col=self.event_col, formula=' + '.join(self.covariates))
        self.summary_df = self.cph.summary

    def summary(self, alpha=0.05):
        print("آزمون مدل خطرات متناسب کاکس (Cox PH model)")
        print("خلاصه ضرایب:")
        for cov in self.covariates:
            row = self.summary_df.loc[cov]
            coef = row['coef']
            hr = np.exp(coef)
            se = row['se(coef)']
            p = row['p']
            print(f"  {cov}: coef={coef:.4f}, HR={hr:.4f}, se={se:.4f}, p-value={p:.6f}")
            if p < alpha:
                print(f"    نتیجه: متغیر معنی‌دار است")
            else:
                print(f"    نتیجه: متغیر معنی‌دار نیست")
        # آزمون نسبت درستنمایی کلی
        llf = self.cph.log_likelihood_
        llnull = self.cph._null_log_likelihood_
        lr_stat = -2 * (llnull - llf)
        lr_p = 1 - chi2.cdf(lr_stat, len(self.covariates))
        print(f"\nآزمون نسبت درستنمایی کلی: LR = {lr_stat:.4f}, p-value = {lr_p:.6f}")


# ------------------------------------------------------------
# تحلیل توان (Power analysis)
# ------------------------------------------------------------
class PowerAnalysis:
    def __init__(self, test_type='ttest', **kwargs):
        """
        test_type: نوع آزمون
            'ttest_one': t-test یک نمونه
            'ttest_two': t-test دو نمونه مستقل
            'ttest_paired': t-test زوجی
            'chisquare_gof': کای‌دو نیکویی برازش
        kwargs: پارامترهای توان (effect_size, n, power, alpha) - دو تا از آنها داده شود
        """
        self.test_type = test_type
        self.params = kwargs
        if not STATSMODELS_AVAILABLE:
            raise ImportError("برای تحلیل توان، statsmodels باید نصب باشد: pip install statsmodels")
        self._compute()

    def _compute(self):
        if self.test_type == 'ttest_one':
            power_analysis = TTestPower()
            self.result = power_analysis.solve_power(**self.params)
        elif self.test_type == 'ttest_two':
            power_analysis = TTestIndPower()
            self.result = power_analysis.solve_power(**self.params)
        elif self.test_type == 'ttest_paired':
            # t-test زوجی مشابه یک نمونه است
            power_analysis = TTestPower()
            self.result = power_analysis.solve_power(**self.params)
        elif self.test_type == 'chisquare_gof':
            power_analysis = GofChisquarePower()
            self.result = power_analysis.solve_power(**self.params)
        else:
            raise ValueError("نوع آزمون نامعتبر")

    def summary(self):
        print("تحلیل توان (Power Analysis)")
        print(f"نوع آزمون: {self.test_type}")
        print("پارامترهای ورودی:")
        for key, value in self.params.items():
            print(f"  {key} = {value}")
        print(f"نتیجه محاسبه: {self.result:.4f}")


# ------------------------------------------------------------
# آزمون نسبت درستنمایی (قبلاً پیاده‌سازی شده، برای کامل بودن)
# ------------------------------------------------------------
class LikelihoodRatioTest:
    def __init__(self, logL_full, logL_reduced, df_diff):
        self.logL_full = logL_full
        self.logL_reduced = logL_reduced
        self.df_diff = df_diff
        self._compute()

    def _compute(self):
        self.LR_stat = -2 * (self.logL_reduced - self.logL_full)
        self.p_value = 1 - chi2.cdf(self.LR_stat, self.df_diff)

    def summary(self, alpha=0.05):
        print("آزمون نسبت درستنمایی (Likelihood Ratio Test)")
        print(f"لگاریتم درستنمایی مدل کامل: {self.logL_full:.4f}")
        print(f"لگاریتم درستنمایی مدل کاهش‌ یافته: {self.logL_reduced:.4f}")
        print(f"آماره LR = {self.LR_stat:.4f}")
        print(f"درجه آزادی = {self.df_diff}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر (مدل کامل برتر است) در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر (مدل کاهش‌یافته کافی است) در سطح {alpha}")